Claims with paid_amount = 0 and claim_amount > ₹10,000

In [0]:
SELECT  
    c.claim_id,
    pt.first_name || ' ' || pt.last_name AS patient_name,
    pr.name AS provider_name,
    pr.specialty,
    c.claim_date,
    c.service_start_date,
    c.diagnosis_code,
    c.procedure_code,
    c.claim_amount,
    c.paid_amount,
    c.claim_status,
    c.submission_mode,
    CASE 
        WHEN c.claim_status = 'Denied' THEN 'Expected - Claim Denied'
        WHEN c.claim_status = 'Pending' THEN 'Review - Claim Pending'
        WHEN c.claim_status = 'Approved' THEN 'ANOMALY - Approved but Zero Payment'
        ELSE 'Unknown Status'
    END AS anomaly_reason
FROM healthcare_cp.hcp.claims_data c
JOIN healthcare_cp.hcp.patients_data pt ON c.patient_id = pt.patient_id
JOIN healthcare_cp.hcp.providers_data pr ON c.provider_id = pr.provider_id
WHERE c.paid_amount = 0 
  AND c.claim_amount > 10000
ORDER BY c.claim_amount DESC;

Billing NPI mismatch between claims and provider metadata

In [0]:
SELECT 
    c.claim_id,
    c.provider_id,
    pr.name AS provider_name,
    pr.specialty,
    pr.npi_number AS provider_npi,
    c.billing_npi AS claim_billing_npi,
    c.claim_date,
    c.claim_amount,
    c.paid_amount,
    c.claim_status,
    pt.first_name || ' ' || pt.last_name AS patient_name
FROM healthcare_cp.hcp.claims_data c
JOIN healthcare_cp.hcp.providers_data pr ON c.provider_id = pr.provider_id
JOIN healthcare_cp.hcp.patients_data pt ON c.patient_id = pt.patient_id
WHERE c.billing_npi != pr.npi_number
  AND c.billing_npi IS NOT NULL
  AND pr.npi_number IS NOT NULL
ORDER BY c.claim_amount DESC;

Same patient, same provider, multiple claims on same day (possible fraud)

In [0]:
WITH same_day_claims AS (
    SELECT 
        patient_id,
        provider_id,
        service_start_date,
        COUNT(claim_id) AS claims_same_day,
        SUM(claim_amount::DECIMAL(10,2)) AS total_amount_same_day,
        LISTAGG(claim_id, ', ') AS claim_ids,
        LISTAGG(procedure_code, ', ') AS procedures
    FROM hcp.claims_data
    WHERE service_start_date IS NOT NULL
    GROUP BY patient_id, provider_id, service_start_date
    HAVING COUNT(claim_id) > 1
)
SELECT 
    sdc.patient_id,
    pt.first_name || ' ' || pt.last_name AS patient_name,
    sdc.provider_id,
    pr.name AS provider_name,
    pr.specialty,
    sdc.service_start_date,
    sdc.claims_same_day,
    sdc.total_amount_same_day,
    sdc.procedures,
    sdc.claim_ids,
    CASE 
        WHEN sdc.claims_same_day >= 5 THEN 'HIGH FRAUD RISK'
        WHEN sdc.claims_same_day >= 3 THEN 'MODERATE FRAUD RISK'
        ELSE 'LOW FRAUD RISK'
    END AS fraud_risk_level
FROM same_day_claims sdc
JOIN hcp.patients_data pt ON sdc.patient_id = pt.patient_id
JOIN hcp.providers_data pr ON sdc.provider_id = pr.provider_id
ORDER BY sdc.claims_same_day DESC, sdc.total_amount_same_day DESC;

Provider with sudden spike in claims after low activity

In [0]:
WITH monthly_provider_activity AS (
    SELECT
        provider_id,
        date_trunc('month', claim_date)::date AS claim_month,
        COUNT(claim_id) AS monthly_claims,
        SUM(claim_amount::decimal(14,2)) AS monthly_billed
    FROM hcp.claims_data
    WHERE claim_date >= current_date - interval '18 months'
    GROUP BY provider_id, date_trunc('month', claim_date)::date
),
provider_activity_analysis AS (
    SELECT
        provider_id,
        claim_month,
        monthly_claims,
        monthly_billed,
        AVG(monthly_claims) OVER (PARTITION BY provider_id
                                  ORDER BY claim_month
                                  ROWS BETWEEN 3 PRECEDING AND 1 PRECEDING) AS avg_prev_3m
    FROM monthly_provider_activity
),
spikes AS (
    SELECT
        *,
        CASE WHEN avg_prev_3m > 0 THEN monthly_claims / avg_prev_3m END AS spike_ratio
    FROM provider_activity_analysis
    WHERE avg_prev_3m IS NOT NULL
)
SELECT
    pr.name AS provider_name,
    pr.specialty,
    pr.years_of_experience,
    to_char(s.claim_month, 'YYYY-MM') AS spike_month,
    round(s.avg_prev_3m, 1) AS avg_prev_3m_claims,
    s.monthly_claims AS spike_month_claims,
    s.monthly_billed AS spike_month_amount,
    round(s.spike_ratio, 2) AS activity_spike_ratio,
    CASE
        WHEN s.spike_ratio >= 10 THEN 'EXTREME SPIKE'
        WHEN s.spike_ratio >= 5 THEN 'HIGH SPIKE'
        WHEN s.spike_ratio >= 3 THEN 'MODERATE SPIKE'
        WHEN s.spike_ratio >= 2 THEN 'LOW SPIKE'
        ELSE 'NORMAL'
    END AS spike_severity
FROM spikes s
JOIN hcp.providers_data pr ON pr.provider_id = s.provider_id
WHERE s.spike_ratio >= 2
ORDER BY s.spike_ratio DESC, s.monthly_claims DESC;